<a href="https://colab.research.google.com/github/funpi89/colab_trasformlearning/blob/master/Transfer_Learning_and_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and notebook gpu setup

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [0]:
!pip install tensorflow-gpu==2.0.0b1

     |████████████████████████████████| 348.9MB 75kB/s 
     |████████████████████████████████| 3.1MB 25.6MB/s 
     |████████████████████████████████| 501kB 46.1MB/s 


In [0]:
!pip install tqdm

# Downloading the Dogs vs Cats dataset

In [0]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2019-09-19 13:03:30--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  56.2MB/s    in 1.2s    

2019-09-19 13:03:32 (56.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



# Import libraries

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-beta1'

# Unzipping the Dogs vs Cats dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

# Setting up dataset paths

In [0]:
dataset_path_new = './cats_and_dogs_filtered/'
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

# Building the model

In [0]:
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 12s 1us/step


In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
# freeze the base_model
base_model.trainable = False

In [0]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

# Compile the model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Creating Data Generators

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [0]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 1000 images belonging to 2 classes.


# Training the model

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
16/16 [==============================] - 20s 1s/step - loss: 0.5778 - accuracy: 0.6930 - val_loss: 0.4803 - val_accuracy: 0.7600
Epoch 2/5
16/16 [==============================] - 8s 529ms/step - loss: 0.5342 - accuracy: 0.7340 - val_loss: 0.4353 - val_accuracy: 0.8040
Epoch 3/5
16/16 [==============================] - 10s 595ms/step - loss: 0.4947 - accuracy: 0.7740 - val_loss: 0.4015 - val_accuracy: 0.8240
Epoch 4/5
16/16 [==============================] - 10s 597ms/step - loss: 0.4589 - accuracy: 0.8050 - val_loss: 0.3666 - val_accuracy: 0.8480
Epoch 5/5
16/16 [==============================] - 9s 592ms/step - loss: 0.4298 - accuracy: 0.8335 - val_loss: 0.3469 - val_accuracy: 0.8650


# Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print('Accuracy after transfer learning : {}'.format(valid_accuracy))

Accuracy after transfer learning : 0.8650000095367432


# Fine Tuning

In [0]:
# unfreeze a few top layers from the model
base_model.trainable = True
len(base_model.layers)

155

In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Compile the model for fine tuning

In [0]:
# after fine tuning, we don't build the model again, just re-compiling
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the model for fine tuning

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 32s 2s/step - loss: 0.1721 - accuracy: 0.9320 - val_loss: 0.0867 - val_accuracy: 0.9650
Epoch 2/5
16/16 [==============================] - 9s 573ms/step - loss: 0.0258 - accuracy: 0.9995 - val_loss: 0.0894 - val_accuracy: 0.9680
Epoch 3/5
16/16 [==============================] - 10s 597ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.0839 - val_accuracy: 0.9690
Epoch 4/5
16/16 [==============================] - 10s 604ms/step - loss: 0.0058 - accuracy: 0.9995 - val_loss: 0.0946 - val_accuracy: 0.9710
Epoch 5/5
16/16 [==============================] - 10s 599ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1021 - val_accuracy: 0.9710


# Fine tuning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print('Accuracy after fine tnuing learning : {}'.format(valid_accuracy))

Accuracy after fine tnuing learning : 0.9710000157356262
